## Day 30 Lecture 2 Assignment

In this assignment, we will learn about random forests. We will use the google play store dataset loaded below.

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [74]:
reviews = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/googleplaystore.csv')

reviews.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In this assignment, you will work more independently. Perform the following steps:
    
1. Select which columns are best suited to predict whether the rating is above 4.5
2. Process the data (including transforming to the correct column type, removing missing values, creating dummy variables, and removing irrelevant variables)
3. Create a random forest model and evaluate
4. Tweak the parameters using grid search to produce a better performing model
5. Show and discuss your results

Good luck!

In [75]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null object
Size              10841 non-null object
Installs          10841 non-null object
Type              10840 non-null object
Price             10841 non-null object
Content Rating    10840 non-null object
Genres            10841 non-null object
Last Updated      10841 non-null object
Current Ver       10833 non-null object
Android Ver       10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [76]:
reviews['Type'].unique()

array(['Free', 'Paid', nan, '0'], dtype=object)

In [0]:
# drop missing values
reviews.dropna(inplace = True, axis = 0)

In [0]:
# dropp irrelevant variables (features)
#Last Updated	Current Ver	Android Ver
reviews.drop(columns = ['App','Category', 'Size', 'Installs', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], inplace = True)


In [79]:
#convert categorical to dummy variables
content_rating_dummy = pd.get_dummies(reviews['Content Rating'], prefix = 'rating', drop_first = True)
type_dummy = pd.get_dummies(reviews['Type'],prefix = 'type', drop_first = True)

reviews = pd.concat([reviews, content_rating_dummy, type_dummy], axis = 1)
reviews.drop(columns= ['Content Rating','Type'], inplace = True)
reviews.head()

,Rating,Reviews,Price,rating_Everyone,rating_Everyone 10+,rating_Mature 17+,rating_Teen,rating_Unrated,type_Paid
0,4.1,159,0,1,0,0,0,0,0
1,3.9,967,0,1,0,0,0,0,0
2,4.7,87510,0,1,0,0,0,0,0
3,4.5,215644,0,0,0,0,1,0,0
4,4.3,967,0,1,0,0,0,0,0


4.4    1108
4.3    1076
4.5    1037
4.2     951
4.6     823
4.1     707
4.0     567
4.7     499
3.9     386
3.8     303
5.0     274
3.7     239
4.8     234
3.6     174
3.5     163
3.4     128
3.3     102
4.9      87
3.0      83
3.1      69
3.2      63
2.9      45
2.8      42
2.6      25
2.7      25
2.5      21
2.3      20
2.4      19
1.0      16
2.2      14
1.9      13
2.0      12
1.7       8
2.1       8
1.8       8
1.6       4
1.4       3
1.5       3
1.2       1
Name: Rating, dtype: int64

In [80]:
reviews['Price'].nunique()
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 9 columns):
Rating                 9360 non-null float64
Reviews                9360 non-null object
Price                  9360 non-null object
rating_Everyone        9360 non-null uint8
rating_Everyone 10+    9360 non-null uint8
rating_Mature 17+      9360 non-null uint8
rating_Teen            9360 non-null uint8
rating_Unrated         9360 non-null uint8
type_Paid              9360 non-null uint8
dtypes: float64(1), object(2), uint8(6)
memory usage: 347.3+ KB


In [0]:
#convert object to numerical (drop $, etc.)

reviews['Price'] = reviews['Price'].str.strip('$')
reviews['Price'] = pd.to_numeric(reviews['Price'], errors = 'coerce', downcast = 'float')
reviews['Reviews'] = pd.to_numeric(reviews['Reviews'], errors = 'coerce', downcast = 'float')

In [82]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 9 columns):
Rating                 9360 non-null float64
Reviews                9360 non-null float32
Price                  9360 non-null float32
rating_Everyone        9360 non-null uint8
rating_Everyone 10+    9360 non-null uint8
rating_Mature 17+      9360 non-null uint8
rating_Teen            9360 non-null uint8
rating_Unrated         9360 non-null uint8
type_Paid              9360 non-null uint8
dtypes: float32(2), float64(1), uint8(6)
memory usage: 274.2 KB


In [0]:
reviews1 = reviews.copy()

reviews1['high_rating'] = np.where(reviews1['Rating'] > 4.5, 1, 0)
reviews1['high_rating'].value_counts()
reviews2 = reviews1.drop(['Rating'], 1)

In [164]:
#Create a random forest model and evaluate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

rfc = RandomForestClassifier(random_state = 21)

X = reviews2.drop('high_rating', axis =1)
y = reviews2['high_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y)

#fit x and y to rfc model 
rfc.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=21, verbose=0,
                       warm_start=False)

In [165]:
cross_val_score(rfc, X_test, y_test, cv=5)

array([0.75266525, 0.7782516 , 0.7542735 , 0.751606  , 0.75802998])

In [166]:
rfc.score(X_train, y_train)

0.9141025641025641

In [0]:
#Tweak the parameters using grid search to produce a better performing model
#Show and discuss your results

In [171]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth': np.arange(1, 21), 'n_estimators': np.arange(10, 50)}

grid = GridSearchCV(rfc, param_grid=params,  scoring='accuracy', cv=3, return_train_score=True)

grid.fit(X_train, y_train)



GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,...
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': array(

In [172]:
grid.score(X_train, y_train)

0.7995726495726496

In [173]:
grid.score(X_test, y_test)

0.8136752136752137

In [151]:
 #find most important features

impor_fea2 = pd.Series(rfc.feature_importances_, index=X.columns).sort_values(ascending = False)

impor_fea2

Reviews                0.945929
Price                  0.041916
type_Paid              0.004951
rating_Mature 17+      0.002429
rating_Everyone        0.001955
rating_Teen            0.001942
rating_Everyone 10+    0.000852
rating_Unrated         0.000027
dtype: float64

In [152]:
#create a list of top 2 features
list_features2 = impor_fea[:2].index.tolist()
list_features2

['Reviews', 'Price']

In [0]:
#create new X with 2 features
X_imp_feat2 = X[list_features]

#re-split and create new model
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_imp_feat2, y)
rfc_imp_feat2 = RandomForestClassifier()

In [154]:
#fit new model
rfc_imp_feat2.fit(X_train2,y_train2)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [155]:
#score test model
rfc_imp_feat2.score(X_test2, y_test2)

0.788034188034188